Bigram and trigram - DTM

In [1]:
import numpy as np
import pandas as pd
import string
import nltk
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from matplotlib import pyplot as plt

In [19]:
#Data - Amazon Review
df =pd.read_excel(r"C:\Users\Lenovo\Desktop\DATA SETS\Amazon.xlsx")

In [20]:
df.head()

,id,asins,brand,categories,colors,dateAdded,dateUpdated,dimension,ean,keys,...,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sizes,upc,weight
0,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I initially had trouble deciding between the p...,"Paperwhite voyage, no regrets!",NaN,NaN,Cristina M,NaN,NaN,205 grams
1,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,Allow me to preface this with a little history...,One Simply Could Not Ask For More,NaN,NaN,Ricky,NaN,NaN,205 grams
2,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,4.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I am enjoying it so far. Great for reading. Ha...,Great for those that just want an e-reader,NaN,NaN,Tedd Gardiner,NaN,NaN,205 grams
3,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I bought one of the first Paperwhites and have...,Love / Hate relationship,NaN,NaN,Dougal,NaN,NaN,205 grams
4,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I have to say upfront - I don't like coroporat...,I LOVE IT,NaN,NaN,Miljan David Tanic,NaN,NaN,205 grams


In [21]:
#Lets take the review_text_columns from the data
text=df[['reviews.text']]
print(text['reviews.text'][0])

I initially had trouble deciding between the paperwhite and the voyage because reviews more or less said the same thing: the paperwhite is great, but if you have spending money, go for the voyage.Fortunately, I had friends who owned each, so I ended up buying the paperwhite on this basis: both models now have 300 ppi, so the 80 dollar jump turns out pricey the voyage's page press isn't always sensitive, and if you are fine with a specific setting, you don't need auto light adjustment).It's been a week and I am loving my paperwhite, no regrets! The touch screen is receptive and easy to use, and I keep the light at a specific setting regardless of the time of day. (In any case, it's not hard to change the setting either, as you'll only be changing the light level at a certain time of day, not every now and then while reading).Also glad that I went for the international shipping option with Amazon. Extra expense, but delivery was on time, with tracking, and I didnt need to worry about cus

In [22]:
df.shape

(1597, 27)

In [23]:
#Step=1 Convert every string into lower case
text['Clean_text'] = text['reviews.text'].str.lower().str.replace('[^a-z ]','')
print(text['Clean_text'][0])

i initially had trouble deciding between the paperwhite and the voyage because reviews more or less said the same thing the paperwhite is great but if you have spending money go for the voyagefortunately i had friends who owned each so i ended up buying the paperwhite on this basis both models now have  ppi so the  dollar jump turns out pricey the voyages page press isnt always sensitive and if you are fine with a specific setting you dont need auto light adjustmentits been a week and i am loving my paperwhite no regrets the touch screen is receptive and easy to use and i keep the light at a specific setting regardless of the time of day in any case its not hard to change the setting either as youll only be changing the light level at a certain time of day not every now and then while readingalso glad that i went for the international shipping option with amazon extra expense but delivery was on time with tracking and i didnt need to worry about customs which i may have if i used a thi

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [27]:
#Remoce the stop words
stop = set(stopwords.words('english'))

def sw(text):
    text=[word for word in text.split() if word not in stop]
    return " ".join(text)
    
#Appplying the sw on clean_text colum

text['Split_words'] = text['Clean_text'].apply(sw)
print(text['Split_words'][0])

initially trouble deciding paperwhite voyage reviews less said thing paperwhite great spending money go voyagefortunately friends owned ended buying paperwhite basis models ppi dollar jump turns pricey voyages page press isnt always sensitive fine specific setting dont need auto light adjustmentits week loving paperwhite regrets touch screen receptive easy use keep light specific setting regardless time day case hard change setting either youll changing light level certain time day every readingalso glad went international shipping option amazon extra expense delivery time tracking didnt need worry customs may used third party shipping service


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [34]:
#Creating the count vectorizer
bigram_count_vect = CountVectorizer(ngram_range=(2,2))
#Fitting the count vect on split words
bigram_count_vect.fit(text['Split_words'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(2, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [35]:
#DTM
Z = bigram_count_vect.fit_transform(text['Split_words'])
Z

<1597x44036 sparse matrix of type '<class 'numpy.int64'>'
	with 128389 stored elements in Compressed Sparse Row format>

In [36]:
#We can control the sparce matrix by min_df and max_df

In [42]:
#Converting it into a two array part of it
#Dtm matrix
DTM_bigram = pd.DataFrame(Z.toarray(),columns=bigram_count_vect.get_feature_names())
DTM_bigram.head()

,aa batteries,abc live,abilities one,abilitieswhen amd,ability check,ability choose,ability connect,ability control,ability create,ability cut,...,zero eye,zero look,zerocons sd,zink happy,zip code,zippy awesome,zombiesi love,zoom work,zooming supported,zumi blues
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
#Greq. table from the DTM
word_cout_freq_bigram = DTM_bigram.sum()
word_cout_freq_bigram
#Convert in into a Data Frame
word_table_bigram = pd.DataFrame(word_cout_freq_bigram).reset_index()
word_table_bigram = word_table_bigram.rename(columns={'index':"Word",0:'Count'})
word_table_bigram.sort_values(by='Count',ascending=False).head()

,Word,Count
19809,kindle fire,485
13656,fire hd,369
13748,fire tv,284
13660,fire hdx,201
20345,last years,184


In [49]:
#Combination of the three words


In [50]:
#Creating the count vectorizer
tigram_count_vect = CountVectorizer(ngram_range=(3,3))
#Fitting the count vect on split words
tigram_count_vect.fit(text['Split_words'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(3, 3), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [51]:
#DTM
Z = tigram_count_vect.fit_transform(text['Split_words'])
Z

<1597x52235 sparse matrix of type '<class 'numpy.int64'>'
	with 130320 stored elements in Compressed Sparse Row format>

In [52]:
#Converting it into a two array part of it
#Dtm matrix
DTM_tigram = pd.DataFrame(Z.toarray(),columns=tigram_count_vect.get_feature_names())
DTM_tigram.head()

,aa batteries duracell,aa batteries ive,abc live tv,abilities one thing,abilitieswhen amd created,ability check email,ability choose want,ability connect issues,ability control home,ability create personal,...,zero eye strain,zero look great,zerocons sd card,zink happy appmy,zip code address,zippy awesome display,zombiesi love came,zoom work great,zooming supported titlesthe,zumi blues clues
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
#Greq. table from the DTM
word_cout_freq_tigram = DTM_tigram.sum()
word_cout_freq_tigram
#Convert in into a Data Frame
word_table_tigram = pd.DataFrame(word_cout_freq_tigram).reset_index()
word_table_tigram = word_table_tigram.rename(columns={'index':"Word",0:'Count'})
word_table_tigram.sort_values(by='Count',ascending=False).head()

,Word,Count
23522,kindle fire hd,238
45009,theyre going fall,118
25291,like theyre going,118
23524,kindle fire hdx,97
51610,would say theyre,90


In [55]:
#TD IDF for Bigram

In [56]:
#Creating the count vectorizer
bigram_tfidf_vect = TfidfVectorizer(ngram_range=(2,2))
#Fitting the count vect on split words
bigram_tfidf_vect.fit(text['Split_words'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [58]:
#DTM
Z = bigram_tfidf_vect.fit_transform(text['Split_words'])
Z

<1597x44036 sparse matrix of type '<class 'numpy.float64'>'
	with 128389 stored elements in Compressed Sparse Row format>

In [60]:
#Converting it into a two array part of it
#Dtm matrix
DTM_bigram_tfidf = pd.DataFrame(Z.toarray(),columns=bigram_tfidf_vect.get_feature_names())
DTM_bigram_tfidf.head()

,aa batteries,abc live,abilities one,abilitieswhen amd,ability check,ability choose,ability connect,ability control,ability create,ability cut,...,zero eye,zero look,zerocons sd,zink happy,zip code,zippy awesome,zombiesi love,zoom work,zooming supported,zumi blues
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
#Greq. table from the DTM
word_tfidf_bigram = DTM_bigram_tfidf.sum()
word_tfidf_bigram
#Convert in into a Data Frame
word_table_bigram_tfidf = pd.DataFrame(word_tfidf_bigram).reset_index()
word_table_bigram_tfidf = word_table_bigram_tfidf.rename(columns={'index':"Word",0:'Score'})
word_table_bigram_tfidf.sort_values(by='Score',ascending=False).head()

,Word,Score
19809,kindle fire,21.049710
13656,fire hd,15.838693
2418,apples buds,13.341446
13748,fire tv,12.459168
1724,amazon prime,9.720496


In [65]:
#TD IDF for Trigram

In [68]:
#Creating the count vectorizer
tigram_tfidf_vect = TfidfVectorizer(ngram_range=(3,3))
#Fitting the count vect on split words
tigram_tfidf_vect.fit(text['Split_words'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(3, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [69]:
#DTM
Z = tigram_tfidf_vect.fit_transform(text['Split_words'])
Z

<1597x52235 sparse matrix of type '<class 'numpy.float64'>'
	with 130320 stored elements in Compressed Sparse Row format>

In [70]:
#Converting it into a two array part of it
#Dtm matrix
DTM_tigram_tfidf = pd.DataFrame(Z.toarray(),columns=tigram_tfidf_vect.get_feature_names())
DTM_tigram_tfidf.head()

,aa batteries duracell,aa batteries ive,abc live tv,abilities one thing,abilitieswhen amd created,ability check email,ability choose want,ability connect issues,ability control home,ability create personal,...,zero eye strain,zero look great,zerocons sd card,zink happy appmy,zip code address,zippy awesome display,zombiesi love came,zoom work great,zooming supported titlesthe,zumi blues clues
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
#Greq. table from the DTM
word_tfidf_tigram = DTM_tigram_tfidf.sum()
word_tfidf_tigram
#Convert in into a Data Frame
word_table_tigram_tfidf = pd.DataFrame(word_tfidf_tigram).reset_index()
word_table_tigram_tfidf = word_table_tigram_tfidf.rename(columns={'index':"Word",0:'Score'})
word_table_tigram_tfidf.sort_values(by='Score',ascending=False).head()

,Word,Score
23522,kindle fire hd,11.903682
45009,theyre going fall,9.070348
25291,like theyre going,9.070348
26886,magnets tangle free,6.956963
51610,would say theyre,6.080781


In [2]:
df = "#Analytics-Knowledge is a great skill to learn @data science"

In [5]:
#Step=1 Convert every string into lower case
df1 =df.str.lower().str.replace('[^a-z ]','')
print(df1)

AttributeError: 'str' object has no attribute 'str'